In [1]:
from queue import PriorityQueue

class Node:
    def __init__(self, parent, state):
        self.state = state  # (x, y) coordinates
        self.parent = parent
        self.g = 0  # Cost to reach this node
        self.h = 0  # Heuristic cost to goal
        self.f = 0  # f = g + h

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return self.state == other.state

def heuristic_func(node, goal):
    # Manhattan distance heuristic
    return abs(goal[0] - node.state[0]) + abs(goal[1] - node.state[1])

def Best_F_Search(maze, start, goals):
    rows = len(maze)
    cols = len(maze[0])
    Total_path = []

    while goals:
        frontier = PriorityQueue()
        start_node = Node(None, start)
        frontier.put((0, start_node))
        explored = set()  # Reset explored nodes for each goal

        goal_found = None  # Track which goal is found
        while not frontier.empty():
            node_f, node = frontier.get()

            # Check if node is one of the goals
            if node.state in goals:
                goal_found = node.state
                path = []
                while node:
                    path.append(node.state)
                    node = node.parent
                Total_path.append(path[::-1])  # Reverse path
                break  # Stop searching for this goal

            explored.add(node.state)

            # Explore neighbors (up, down, left, right)
            for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                new_pos = (node.state[0] + x, node.state[1] + y)

                if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in explored:
                    child = Node(node, new_pos)
                    child.h = min(heuristic_func(child, g) for g in goals)  # Heuristic to the closest goal
                    child.g = node.g + 1
                    child.f = child.g + child.h
                    frontier.put((child.f, child))

        if goal_found:
            goals.remove(goal_found)  # Remove only the found goal
            start = goal_found  # Update start position

    return Total_path if Total_path else None

# Test the search algorithm
maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]
start = (0, 0)
goals = [(4, 4), (3, 3)]

paths = Best_F_Search(maze, start, goals)
if paths:
    print("Paths found:", paths)
else:
    print("No path found")


Paths found: [[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3)], [(3, 3), (3, 4), (4, 4)]]


In [2]:
import heapq
import random
import threading
import time

class Node:
    def __init__(self, parent, state):
        self.state = state  # (x, y) coordinates
        self.parent = parent
        self.g = float('inf')  # Cost from start node
        self.h = 0  # Heuristic cost to goal
        self.f = float('inf')  # f = g + h

    def __lt__(self, other):
        return self.f < other.f

def heuristic(node, goal):
    """Manhattan distance heuristic."""
    return abs(goal.state[0] - node.state[0]) + abs(goal.state[1] - node.state[1])

def initialize_costs(maze):
    """Initialize random edge costs for a given maze."""
    rows, cols = len(maze), len(maze[0])
    for r in range(rows):
        for c in range(cols):
            if maze[r][c] == 0:  # Only set costs for walkable cells
                for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] == 0:
                        cost_map[((r, c), (nr, nc))] = random.randint(1, 10)

def update_costs():
    """Randomly update edge costs at intervals."""
    while True:
        time.sleep(5)  # Update every 5 seconds
        edge = random.choice(list(cost_map.keys()))
        new_cost = random.randint(1, 10)
        cost_map[edge] = new_cost
        print(f"Updated edge {edge} cost to {new_cost}")

def astar_search(maze, start, goal):
    """A* search with dynamic costs."""
    frontier = []
    heapq.heappush(frontier, (0, Node(None, start)))
    explored = {}
    goal_node = Node(None, goal)

    while frontier:
        _, node = heapq.heappop(frontier)
        if node.state == goal:
            path = []
            while node:
                path.append(node.state)
                node = node.parent
            return path[::-1]

        explored[node.state] = node.g

        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_pos = (node.state[0] + dr, node.state[1] + dc)

            if 0 <= new_pos[0] < len(maze) and 0 <= new_pos[1] < len(maze[0]) and maze[new_pos[0]][new_pos[1]] == 0:
                cost = cost_map.get((node.state, new_pos), float('inf'))
                new_g = node.g + cost

                if new_pos not in explored or new_g < explored[new_pos]:
                    child = Node(node, new_pos)
                    child.g = new_g
                    child.h = heuristic(child, goal_node)
                    child.f = child.g + child.h
                    heapq.heappush(frontier, (child.f, child))

    return None  # No path found

# Maze representation (0 = walkable, 1 = obstacle)
maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

# Initialize cost map
cost_map = {}
initialize_costs(maze)

# Start dynamic cost updates
threading.Thread(target=update_costs, daemon=True).start()

# Run A* search
start, goal = (0, 0), (4, 4)
path = astar_search(maze, start, goal)

if path:
    print("Path found:", path)
else:
    print("No path found")


Path found: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]


In [3]:
import heapq
import random

class DeliveryPoint:
    def __init__(self, location, deadline):
        self.location = location  # (x, y) coordinates
        self.deadline = deadline  # Time window constraint

    def __lt__(self, other):
        return self.deadline < other.deadline  # Prioritize earlier deadlines

def heuristic(a, b):
    """Manhattan distance heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def greedy_best_first_search(start, deliveries):
    """Greedy Best-First Search to optimize delivery routes with time windows."""
    frontier = []
    heapq.heappush(frontier, (0, start))  # Start from the warehouse
    visited = set()
    path = []
    time_elapsed = 0

    while frontier and deliveries:
        _, current = heapq.heappop(frontier)

        if current in visited:
            continue
        visited.add(current)

        # Find the nearest delivery with the strictest deadline
        nearest_delivery = min(deliveries, key=lambda d: (d.deadline, heuristic(current, d.location)))

        travel_time = heuristic(current, nearest_delivery.location)
        arrival_time = time_elapsed + travel_time

        if arrival_time <= nearest_delivery.deadline:  # Ensure we meet the time window
            time_elapsed = arrival_time
            path.append(nearest_delivery.location)
            deliveries.remove(nearest_delivery)
            heapq.heappush(frontier, (heuristic(nearest_delivery.location, start), nearest_delivery.location))

    return path if not deliveries else None  # Return path if all deliveries were completed

# Example: Delivery points with time windows
deliveries = [
    DeliveryPoint((2, 3), 10),  # Needs to be delivered by time = 10
    DeliveryPoint((4, 4), 15),
    DeliveryPoint((1, 2), 8),
    DeliveryPoint((3, 1), 12)
]

start = (0, 0)  # Warehouse location

# Find optimized delivery path
path = greedy_best_first_search(start, deliveries)

if path:
    print("Optimized Delivery Path:", path)
else:
    print("Unable to complete all deliveries on time.")


Optimized Delivery Path: [(1, 2), (2, 3), (3, 1), (4, 4)]
